# Simulate canonical forms

We only simulate computation of CF on $k\times (n-k)$ matrices

Choose parameters: $q$, $k$ and $n$

In [1]:
q = 5
k = 3
n = 7

Fq = GF(q)

NameError: name 'GF' is not defined

# Case 3

We sample a random matrix $\mathbf A\in \mathbb F_q^{k\times(n-k)}$, then sample:

- a permutation matrix $\mathbf P_r\in S_k$

- a permutation matrix $\mathbf P_c\in S_{n-k}$

We set $\mathbf A' = \mathbf P_r\cdot \mathbf A \cdot \mathbf P_c$ and verify that

$\mathsf{CF}(\mathbf A) = \mathsf{CF}(\mathbf A')$

In [59]:
#Works only if q is a prime
#Returns the index corresponding to the minimum multiset
#Return -1 if the multisets are the same
def lex_min_multisets(a_multiset, b_multiset):
    
    for i in range(len(a_multiset)):
        if a_multiset[i] < b_multiset[i]:
            return 0
        else:
            if a_multiset[i] > b_multiset[i]:
                return 1
            
    #At this point, this means the two multisets are the same: return -1
    return -1

#Works only if q is a prime
#Returns the index corresponding to the minimum multiset
#If the vectors are the same, return the first one by default
def lex_min_vectors(a_multiset, b_multiset):
    
    for i in range(len(a_multiset)):
        if a_multiset[i] < b_multiset[i]:
            return 0
        else:
            if a_multiset[i] > b_multiset[i]:
                return 1
            
    #At this point, this means the two multisets are the same: return 0
    return 0

#Sort multisets using lexicograph ordering
#Raises an error if, at some point, the multisets are equal
#The function returns the permutation sorting rows
def sort_multisets(row_multisets):
    
    #Representation of permutation as list of indices
    indices = [i for i in range(len(row_multisets))]
    
    swap = True
    while swap: #Loop until swap is true
        
        swap = False
        for i in range(len(row_multisets)-1):

            min_index = lex_min_multisets(row_multisets[i], row_multisets[i+1]) #lex ordering of multisets
     
            
            #Report failure if min_index == -1
            if min_index == -1:
                return -1
            
            #Swap elements, if needed
            if min_index == 1:

                swap = True #specify that a swap was done
                    
                #Swap multisets
                tmp = row_multisets[i+1]
                row_multisets[i+1] = row_multisets[i]
                row_multisets[i] = tmp
                
                #Swap indices
                tmp = indices[i+1]
                indices[i+1] = indices[i]
                indices[i] = tmp
                
    return indices

#Sort vectors using lexicograph ordering
#The function returns the permutation sorting columns
def sort_vectors(vectors):
    
    #Representation of permutation as list of indices
    indices = [i for i in range(len(vectors))]
    
    swap = True
    while swap: #Loop until swap is true
        
        swap = False
        for i in range(len(vectors)-1):
            min_index = lex_min_vectors(vectors[i], vectors[i+1]) #lex ordering of vectors
                        
            #Swap elements, if needed
            if min_index == 1:

                swap = True #specify that a swap was done
                    
                #Swap multisets
                tmp = vectors[i+1]
                vectors[i+1] = vectors[i]
                vectors[i] = tmp
                
                #Swap indices
                tmp = indices[i+1]
                indices[i+1] = indices[i]
                indices[i] = tmp
                
    return indices

#To order rows and columns, we use a schoolbook algorithm with cost O(n^2)
#The code works only if q is prime
def case_3_CF(B):
    
    Fq = B[0,0].parent()
        
    #Compute multisets of rows
    row_multisets = []
    for i in range(B.nrows()):
        b_i = B[i,:].change_ring(ZZ).list()
        b_i.sort()
        row_multisets.append(b_i)

    #Compute row permutation sorting rows 
    row_indices = sort_multisets(row_multisets)
    
    #Report failure if row permutation is not defined
    if row_indices == -1:
        return -1, -1, -1
    
    #Apply row permutation
    row_sorted_B = matrix(Fq, B.nrows(), B.ncols())
    for i in range(len(row_indices)):
        row_sorted_B[i,:] = B[jjj[i],:]
    
    #Now, sort columns
    vectors = [row_sorted_B[:,i].list() for i in range(B.ncols())]
    
    #Compute permutation sorting columns
    col_indices = sort_vectors(vectors)
    
    #Apply column permutation
    CF_B = matrix(Fq, B.nrows(), B.ncols())
    for i in range(len(col_indices)):
        CF_B[:,i] = row_sorted_B[:,col_indices[i]]
        
    return row_indices, col_indices, CF_B

Generate matrices

In [65]:
A = random_matrix(Fq, k, n-k)

#Sample permutations
Pr = Permutations(k).random_element().to_matrix().change_ring(Fq)
Pc = Permutations(n-k).random_element().to_matrix().change_ring(Fq)

A_prime = Pr*A*Pc
print("A =")
print(A)
print(" ")
print("A' =")
print(A_prime)

A =
[0 1 1 1]
[2 0 1 2]
[2 1 0 0]
 
A' =
[0 1 1 1]
[2 1 0 2]
[2 0 1 0]


Compute canonical forms, showing step-by-step operations

In [66]:
row_perm, col_perm, CF_A = case_3_CF(A)
if row_perm == -1:
    print("CF(A) is not well defined!")
else:
    print("CF(A) = ")
    print(CF_A)

    print("  ")

    print("Showing step-by-step operations:")
    print(" ")

    print("1) Row permutation = ")
    print(row_perm)
    print(" ")

    #Construct permutation matrix out of row_perm
    S_k = Permutations(k)
    Pr = S_k([row_perm[i]+1 for i in range(k)])
    print("2) Applying row permutation to A")
    print(Pr.to_matrix().transpose()*A)
    print(" ")

    print("3) Column permutation = ")
    print(col_perm)
    print(" ")

    #Construct permutation matrix out of row_perm
    S_n_minus_k = Permutations(n-k)
    Pc = S_n_minus_k([col_perm[i]+1 for i in range(n-k)])
    print("4) Applying also column permutation")
    print(Pr.to_matrix().transpose()*A*Pc.to_matrix())

CF(A) = 
[0 0 1 2]
[1 1 1 0]
[1 2 0 2]
  
Showing step-by-step operations:
 
1) Row permutation = 
[2, 0, 1]
 
2) Applying row permutation to A
[2 1 0 0]
[0 1 1 1]
[2 0 1 2]
 
3) Column permutation = 
[2, 3, 1, 0]
 
4) Applying also column permutation
[0 0 1 2]
[1 1 1 0]
[1 2 0 2]


Now, do the same to $\mathbf A'$ and verify the canonical forms are equal

In [67]:
row_perm_prime, col_perm_prime, CF_A_prime = case_3_CF(A_prime)
print("CF(A') = ")
print(CF_A_prime)
print(" ")

print("CF(A) == CF(A')?", CF_A == CF_A_prime)

CF(A') = 
[0 0 1 2]
[1 1 1 0]
[1 2 0 2]
 
CF(A) == CF(A')? True


# Case 4 (to be done)

In [119]:
A_prime = copy(A)
for i in range(k):
    s = sum(A[i,j] for j in range(n-k))
    s_prime = sum(A[i,j]^(q-2) for j in range(n-k))
    print(s, s_prime)
    if s != 0:
        A_prime[i,:] = s^-1*A_prime[i,:]
        print("---> = ", s^-1)
    else:
        if s_prime != 0:
            A_prime[i,:] = s_prime*A_prime[i,:]
            print("---> = ", s_prime)
        else:
            print("!!! ERROR !!!")

2 2
---> =  3
0 1
---> =  1
3 2
---> =  2


In [120]:
tilde_A_prime = copy(tilde_A)
for i in range(k):
    s = sum(tilde_A[i,j] for j in range(n-k))
    s_prime = sum(tilde_A[i,j]^(q-2) for j in range(n-k))
    print(s, s_prime)
    if s != 0:
        tilde_A_prime[i,:] = s^-1*tilde_A_prime[i,:]
        print("---> = ", s^-1)
    else:
        if s_prime != 0:
            tilde_A_prime[i,:] = s_prime*tilde_A_prime[i,:]
            print("---> = ", s_prime)
        else:
            print("!!! ERROR !!!")

0 1
---> =  1
3 3
---> =  2
4 4
---> =  4


In [121]:
print("A_prime = :")
print(A_prime)
print(" ")
print("tilde_A_prime = :")
print(tilde_A_prime)
print(" ")

print("Are perms ok?",Pr*A_prime*Pc == tilde_A_prime)

A_prime = :
[1 4 3 3]
[2 1 1 1]
[2 0 1 3]
 
tilde_A_prime = :
[1 1 1 2]
[3 4 3 1]
[1 0 3 2]
 
Are perms ok? True


In [ ]:
A = random_matrix(Fq, k, n-k)

#sampling diagonal and monomial
Dr = matrix(Fq, k, k)
for i in range(k):
    a = Fq.random_element()
    while a == 0:
        a = Fq.random_element()
    Dr[i,i] = a
Pr = Permutations(k).random_element().to_matrix().change_ring(Fq)
Pc = Permutations(n-k).random_element().to_matrix().change_ring(Fq)

tilde_A = Pr*Dr*A*Pc
print("A =")
print(A)
print(" ")
print("tilde_A =")
print(tilde_A)